## yolo test

```bash
uv add "onnx>=1.12.0,<1.18.0" "onnxslim>=0.1.53" "onnxruntime"
```

use this to train additional images ...

```python
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
```

In [1]:
import onnx
import onnxruntime
import onnxslim

print("onnx version: ", onnx.__version__)
print("onnxruntime version: ", onnxruntime.__version__)
print("onnxsim version: ", onnxslim.__version__)


onnx version:  1.17.0
onnxruntime version:  1.22.0
onnxsim version:  0.1.53


In [2]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')
# model.export(format='onnx', simplify=True)

In [3]:
results = model("bailey.jpeg")
results[0].show()


image 1/1 /Users/Gerry.Wagner/develop/psueng/sweng-894-capstone/python/bailey.jpeg: 640x480 1 dog, 1 couch, 1 remote, 30.7ms
Speed: 1.3ms preprocess, 30.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)
